<a href="https://colab.research.google.com/github/c-marq/cap4767-data-mining/blob/main/solutions/labs/lab01_forecasting_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1 — SOLUTION KEY 🔑
## Time Series Exploration & Forecasting
**CAP4767 Data Mining with Python** | Miami Dade College — Kendall Campus

**Covers:** Chapters 1–2 (Rolling Windows, Resampling, Decomposition, SARIMAX, Prophet)

**Points:** 50 | **Due:** See Canvas for deadline | **Submission:** Download as .ipynb and upload to Canvas

**Dataset:** Florida Hotel Occupancy — quarterly data from 2005–2024 (80 observations, 10 columns). You will choose **one numeric column** as your forecasting target.

| Part | Skills Tested | Points |
|------|--------------|--------|
| A: Exploration (Week 1 skills) | Rolling windows, resampling, decomposition | 20 |
| B: Forecasting (Week 2 skills) | SARIMAX, Prophet, comparison | 20 |
| C: Reflection | Written analysis | 10 |

<div style="background-color: #D6EAF8; border-left: 5px solid #2E86C1; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1A5276;">💡 GRADING PHILOSOPHY</strong><br>
  This lab rewards <strong>process over perfection</strong>. If your code doesn't work but you explain what you tried and what went wrong, you earn most of the points. A student who writes "I tried X, it failed because Y, so I adjusted to Z" earns more than one who submits broken code with no explanation.
</div>

### Student Information

- **Name:** SOLUTION KEY
- **Date:** Spring 2026
- **Target Column Chosen:** avg_daily_rate_usd

---
## Setup

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Run both cells below. Do not modify them.
</div>

In [ ]:
# ============================================================
# Setup — Run this cell first. Do not modify.
# ============================================================
!pip install -q pmdarima prophet

In [ ]:
# ============================================================
# Imports & Data Loading — Run this cell. Do not modify.
# ============================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings, logging

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import auto_arima
from prophet import Prophet
from sklearn.metrics import mean_squared_error, r2_score

warnings.filterwarnings("ignore")
logging.getLogger("prophet").setLevel(logging.WARNING)
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
plt.rcParams["figure.figsize"] = (12, 5)
plt.rcParams["figure.dpi"] = 100

# Load data
data_url = "https://raw.githubusercontent.com/c-marq/cap4767-data-mining/refs/heads/main/data/florida_hotel_occupancy.csv"
hotel_df = pd.read_csv(data_url, parse_dates=["quarter_start"], index_col="quarter_start")

print(f"Dataset: {hotel_df.shape[0]} rows × {hotel_df.shape[1]} columns")
print(f"\nAvailable columns:")
for col in hotel_df.columns:
    print(f"  • {col} — range: {hotel_df[col].min():.1f} to {hotel_df[col].max():.1f}")

---
## Choose Your Target Column

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Pick <strong>one</strong> numeric column from the dataset as your forecasting target. Set it in the cell below. Do NOT use <code>occupancy_rate_pct</code> — that was the group exercise target.
</div>

<div style="background-color: #FEF9E7; border-left: 5px solid #F1C40F; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #7D6608;">⚠️ COMMON MISTAKE</strong><br>
  Make sure to set the frequency with <code>.asfreq("QS")</code> after extracting your column. Without this, SARIMAX and Prophet will not know the data is quarterly.
</div>

In [ ]:
# Choose your target column (change the string below)
TARGET_COLUMN = "avg_daily_rate_usd"  # ← CHANGE THIS to your chosen column

# Extract as time series with quarterly frequency
ts_data = hotel_df[TARGET_COLUMN].asfreq("QS")

print(f"Target: {TARGET_COLUMN}")
print(f"Observations: {len(ts_data)}")
print(f"Range: {ts_data.min():.2f} to {ts_data.max():.2f}")
print(f"Mean: {ts_data.mean():.2f}")

---
# Part A: Time Series Exploration (20 points)

Apply Week 1 skills to explore your chosen time series.

### Task A1: Time Series Plot (4 points)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Create a line plot of your target variable over time. Include a descriptive title, axis labels, and a grid.
</div>

In [ ]:
# A1: Time series plot
plt.figure(figsize=(12, 5))
plt.plot(ts_data.index, ts_data.values, color="steelblue", linewidth=1.5)
plt.title(f"Florida Hotels — {TARGET_COLUMN} (Quarterly, 2005–2024)")
plt.xlabel("Quarter")
plt.ylabel(TARGET_COLUMN)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Task A2: Rolling Windows (4 points)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Calculate a 4-quarter rolling mean and a 4-quarter rolling standard deviation. Plot both on the same chart as the original data (3 lines total).
</div>

In [ ]:
# A2: Rolling windows
rolling_mean = ts_data.rolling(window=4).mean()
rolling_std = ts_data.rolling(window=4).std()

plt.figure(figsize=(12, 5))
plt.plot(ts_data.index, ts_data.values, label="Original", color="steelblue", alpha=0.6)
plt.plot(rolling_mean.index, rolling_mean.values, label="4-Quarter Rolling Mean", color="darkorange", linewidth=2)
plt.plot(rolling_std.index, rolling_std.values, label="4-Quarter Rolling Std", color="green", linewidth=2)
plt.title(f"Rolling Windows — {TARGET_COLUMN}")
plt.xlabel("Quarter")
plt.ylabel(TARGET_COLUMN)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #922B21;">🛑 STOP AND CHECK — Part A Checkpoint 1</strong><br>
  <ul>
    <li>Your time series plot shows 80 data points from 2005 to 2024</li>
    <li>The rolling mean line is smoother than the original — it filters out seasonal noise</li>
    <li>The rolling std line shows whether volatility is increasing or decreasing over time</li>
  </ul>
</div>

### Task A3: Resampling (4 points)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Resample your quarterly data to <strong>annual frequency</strong> using the mean. Plot the annual version alongside the quarterly original.
</div>

In [ ]:
# A3: Resample to annual
annual = ts_data.resample("YS").mean()

plt.figure(figsize=(12, 5))
plt.plot(ts_data.index, ts_data.values, label="Quarterly", color="steelblue", alpha=0.5)
plt.plot(annual.index, annual.values, label="Annual Mean", color="darkorange", linewidth=2, marker="o")
plt.title(f"Quarterly vs Annual — {TARGET_COLUMN}")
plt.xlabel("Year")
plt.ylabel(TARGET_COLUMN)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Task A4: ADF Stationarity Test (4 points)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Run the Augmented Dickey-Fuller test. Print the test statistic, p-value, and a conclusion about stationarity. Then write 1–2 sentences explaining what the result means for forecasting.
</div>

In [ ]:
# A4: ADF test
adf_result = adfuller(ts_data.dropna(), autolag="AIC")

print(f"ADF Statistic: {adf_result[0]:.4f}")
print(f"P-value:       {adf_result[1]:.4f}")
print()
if adf_result[1] < 0.05:
    print("✅ Data IS stationary (p < 0.05)")
else:
    print("⚠️  Data is NOT stationary (p ≥ 0.05) — SARIMAX will need differencing")

**Your interpretation (1–2 sentences):**

**Sample answer:** The ADF test p-value indicates whether the average daily rate has a statistically significant trend over time. If non-stationary (p ≥ 0.05), this means rates have been trending upward over the 20-year period, which makes sense given inflation and rising tourism demand. SARIMAX will handle this through differencing (the `d` parameter), while Prophet models the trend directly.

### Task A5: Seasonal Decomposition (4 points)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Run <code>seasonal_decompose</code> with <code>period=4</code>. Display the decomposition plot. Then write 1–2 sentences describing the seasonal pattern you see.
</div>

In [ ]:
# A5: Seasonal decomposition
decomposition = seasonal_decompose(ts_data, model="additive", period=4)
fig = decomposition.plot()
fig.set_size_inches(12, 8)
plt.tight_layout()
plt.show()

**Your interpretation (1–2 sentences):**

**Sample answer:** The decomposition reveals a strong seasonal pattern with Q1 consistently showing the highest values (driven by snowbird season and peak winter tourism) and Q3/Q4 showing seasonal lows. The trend component shows steady growth with a visible dip around 2020 from the COVID-19 pandemic, followed by a sharp recovery that pushed rates above pre-pandemic levels.

---
# Part B: Forecasting (20 points)

Apply Week 2 skills: train/test split, SARIMAX, Prophet, and comparison.

### Task B1: Train/Test Split (4 points)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Split the data: first 64 quarters for training, remaining 16 for testing. Print the date ranges for each set and create a visualization showing the split.
</div>

In [ ]:
# B1: Train/test split
train = ts_data.iloc[:64]
test = ts_data.iloc[64:]

print(f"Train: {len(train)} quarters ({train.index[0].year}–{train.index[-1].year})")
print(f"Test:  {len(test)} quarters ({test.index[0].year}–{test.index[-1].year})")

plt.figure(figsize=(12, 5))
plt.plot(train.index, train, label="Train", color="steelblue")
plt.plot(test.index, test, label="Test", color="darkorange")
plt.axvline(x=test.index[0], color="red", linestyle="--", alpha=0.7, label="Split")
plt.title(f"Train/Test Split — {TARGET_COLUMN}")
plt.xlabel("Quarter")
plt.ylabel(TARGET_COLUMN)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Task B2: SARIMAX Forecast (4 points)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Use <code>auto_arima(train, seasonal=True, m=4)</code> to find parameters, then fit SARIMAX and forecast the test period. Print the parameters, RMSE, and R². Plot the forecast against actuals.
</div>

In [ ]:
# B2: SARIMAX
auto_model = auto_arima(train, seasonal=True, m=4, suppress_warnings=True,
                        error_action="ignore", trace=False)

order = auto_model.order
seasonal_order = auto_model.seasonal_order
print(f"Best order: {order}")
print(f"Seasonal order: {seasonal_order}")
print(f"AIC: {auto_model.aic():.2f}")

# Fit and forecast
sarimax_fit = SARIMAX(train,
    order=order, seasonal_order=seasonal_order,
    enforce_stationarity=False, enforce_invertibility=False
).fit(disp=False)
sarimax_forecast = sarimax_fit.forecast(steps=len(test))

sarimax_rmse = np.sqrt(mean_squared_error(test, sarimax_forecast))
sarimax_r2 = r2_score(test, sarimax_forecast)
print(f"\nSARIMAX RMSE: {sarimax_rmse:.2f}")
print(f"SARIMAX R²:   {sarimax_r2:.4f}")

# Plot
plt.figure(figsize=(12, 5))
plt.plot(train.index, train, label="Train", color="steelblue")
plt.plot(test.index, test, label="Actual", color="darkorange")
plt.plot(test.index, sarimax_forecast, label="SARIMAX Forecast", linestyle="--", color="green")
plt.axvline(x=test.index[0], color="red", linestyle="--", alpha=0.3)
plt.title(f"SARIMAX Forecast — {TARGET_COLUMN} (RMSE={sarimax_rmse:.2f}, R²={sarimax_r2:.4f})")
plt.xlabel("Quarter")
plt.ylabel(TARGET_COLUMN)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

<div style="background-color: #FADBD8; border-left: 5px solid #E74C3C; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #922B21;">🛑 STOP AND CHECK — Part B Checkpoint</strong><br>
  <ul>
    <li>auto_arima returned a set of parameters (p,d,q)(P,D,Q,4)</li>
    <li>RMSE is a reasonable number (not zero, not astronomically large)</li>
    <li>The forecast line roughly follows the actual seasonal pattern</li>
  </ul>
  If your forecast is flat, check that <code>m=4</code> is set in <code>auto_arima</code>.
</div>

### Task B3: Prophet Forecast (4 points)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Fit a Prophet model on the training data with <code>seasonality_mode="multiplicative"</code>. Forecast the test period. Print RMSE and R². Plot the forecast against actuals. Display the component plots.
</div>

<div style="background-color: #FEF9E7; border-left: 5px solid #F1C40F; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #7D6608;">⚠️ COMMON MISTAKE</strong><br>
  Prophet requires columns named exactly <code>ds</code> and <code>y</code>. Create a new DataFrame: <code>pd.DataFrame({"ds": train.index, "y": train.values})</code>
</div>

In [ ]:
# B3: Prophet
train_prophet = pd.DataFrame({"ds": train.index, "y": train.values})

prophet_fit = Prophet(
    changepoint_prior_scale=0.05,
    seasonality_prior_scale=10.0,
    seasonality_mode="multiplicative",
    changepoint_range=0.85,
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False
)
prophet_fit.fit(train_prophet)

future = prophet_fit.make_future_dataframe(periods=len(test), freq="QS")
forecast_df = prophet_fit.predict(future)
prophet_forecast = forecast_df["yhat"].iloc[-len(test):].values

prophet_rmse = np.sqrt(mean_squared_error(test, prophet_forecast))
prophet_r2 = r2_score(test, prophet_forecast)
print(f"Prophet RMSE: {prophet_rmse:.2f}")
print(f"Prophet R²:   {prophet_r2:.4f}")

# Plot
plt.figure(figsize=(12, 5))
plt.plot(train.index, train, label="Train", color="steelblue")
plt.plot(test.index, test, label="Actual", color="darkorange")
plt.plot(test.index, prophet_forecast, label="Prophet Forecast", linestyle="--", color="purple")
plt.axvline(x=test.index[0], color="red", linestyle="--", alpha=0.3)
plt.title(f"Prophet Forecast — {TARGET_COLUMN} (RMSE={prophet_rmse:.2f}, R²={prophet_r2:.4f})")
plt.xlabel("Quarter")
plt.ylabel(TARGET_COLUMN)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Component plots
fig = prophet_fit.plot_components(forecast_df)
plt.tight_layout()
plt.show()

### Task B4: Model Comparison (4 points)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Create a comparison table and a combined overlay plot showing both forecasts on the same chart. Declare a winner.
</div>

In [ ]:
# B4: Comparison
comparison = pd.DataFrame({
    "Metric": ["RMSE", "R²", "Approach", "Stationarity Required?"],
    "SARIMAX": [f"{sarimax_rmse:.2f}", f"{sarimax_r2:.4f}",
                f"Statistical — order {order}", "Yes"],
    "Prophet": [f"{prophet_rmse:.2f}", f"{prophet_r2:.4f}",
                "Decomposable — trend + seasonality", "No"]
})
print(comparison.to_string(index=False))
print()

if sarimax_rmse < prophet_rmse:
    winner = "SARIMAX"
    print(f"🏆 SARIMAX wins by {prophet_rmse - sarimax_rmse:.2f}")
else:
    winner = "Prophet"
    print(f"🏆 Prophet wins by {sarimax_rmse - prophet_rmse:.2f}")

# Combined plot
plt.figure(figsize=(12, 5))
plt.plot(train.index, train, label="Train", color="steelblue", alpha=0.5)
plt.plot(test.index, test, label="Actual", color="darkorange", linewidth=2)
plt.plot(test.index, sarimax_forecast, label=f"SARIMAX (RMSE={sarimax_rmse:.2f})",
         linestyle="--", color="green")
plt.plot(test.index, prophet_forecast, label=f"Prophet (RMSE={prophet_rmse:.2f})",
         linestyle="--", color="purple")
plt.axvline(x=test.index[0], color="red", linestyle="--", alpha=0.3)
plt.title(f"Model Comparison — {TARGET_COLUMN}")
plt.xlabel("Quarter")
plt.ylabel(TARGET_COLUMN)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Task B5: Future Forecast (4 points)

<div style="background-color: #D5F5E3; border-left: 5px solid #27AE60; padding: 15px; margin: 15px 0; border-radius: 4px;">
  <strong style="color: #1E8449;">✅ DO THIS</strong><br>
  Using the <strong>winning model</strong>, retrain on all 80 quarters and forecast <strong>8 quarters into the future</strong> (2025–2026). Plot the full history + future forecast.
</div>

In [ ]:
# B5: Future forecast using winning model
full_data = ts_data.copy()
future_periods = 8

if winner == "SARIMAX":
    final_model = SARIMAX(full_data,
        order=order, seasonal_order=seasonal_order,
        enforce_stationarity=False, enforce_invertibility=False
    ).fit(disp=False)
    future_forecast = final_model.forecast(steps=future_periods)
    future_index = pd.date_range(
        start=full_data.index[-1] + pd.tseries.frequencies.to_offset("QS"),
        periods=future_periods, freq="QS")
    model_name = "SARIMAX"
else:
    full_prophet_df = pd.DataFrame({"ds": full_data.index, "y": full_data.values})
    final_prophet = Prophet(
        changepoint_prior_scale=0.05, seasonality_prior_scale=10.0,
        seasonality_mode="multiplicative", changepoint_range=0.85,
        yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False
    )
    final_prophet.fit(full_prophet_df)
    future_df = final_prophet.make_future_dataframe(periods=future_periods, freq="QS")
    pred = final_prophet.predict(future_df)
    future_forecast = pred["yhat"].iloc[-future_periods:].values
    future_index = pred["ds"].iloc[-future_periods:].values
    model_name = "Prophet"

plt.figure(figsize=(12, 5))
plt.plot(full_data.index, full_data, label="Historical", color="steelblue")
plt.plot(future_index, future_forecast, label=f"{model_name} Forecast (2025–2026)",
         linestyle="--", color="crimson", linewidth=2, marker="o")
plt.axvline(x=full_data.index[-1], color="red", linestyle="--", alpha=0.5, label="Forecast Start")
plt.title(f"8-Quarter Future Forecast — {TARGET_COLUMN} ({model_name})")
plt.xlabel("Quarter")
plt.ylabel(TARGET_COLUMN)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nFuture forecast ({model_name}):")
for dt, val in zip(future_index, future_forecast):
    print(f"  {pd.Timestamp(dt).strftime('%Y Q%q') if hasattr(pd.Timestamp(dt), 'quarter') else dt}: {val:.2f}")

---
# Part C: Reflection (10 points)

### C1: Model Analysis (5 points)

In 3–5 sentences, answer: **Why do you think the winning model performed better on your chosen variable?** Consider the characteristics of your target column (trend strength, seasonal amplitude, COVID disruption, volatility) and how each model handles those features.

**Your answer:**

**Sample answer:** The winning model likely performed better because the average daily rate has a strong, consistent upward trend with relatively stable seasonal amplitude. SARIMAX excels when the seasonal pattern is regular and the data is well-behaved after differencing. Prophet's strength is handling irregular changepoints and holiday effects — but Florida hotel rates follow a predictable seasonal cycle without many sudden structural breaks (except COVID). The COVID dip and recovery created a challenging test set for both models, but the model that better captured the post-pandemic rate surge earned the lower RMSE.

### C2: Real-World Application (5 points)

In 3–5 sentences, answer: **If you were presenting this forecast to a hotel executive in Miami, what caveats or limitations would you mention?** Think about: sample size, external events, model assumptions, and the difference between the test period and the future.

**Your answer:**

**Sample answer:** I would caution the executive that our model was trained on 80 quarterly observations — enough to capture patterns, but not enough to have seen multiple black-swan events. The COVID period in our training data is unprecedented and may skew the model's understanding of 'normal.' Future forecasts assume the same seasonal pattern continues, but factors like new hotel construction, changes in international travel policy, or another major disruption could invalidate the forecast. I'd recommend treating the 8-quarter forecast as a baseline scenario and updating it quarterly as new data becomes available.

---
## Troubleshooting

| Problem | Likely Cause | Fix |
|---------|-------------|-----|
| `ModuleNotFoundError: prophet` | Package didn't install | Re-run the install cell; restart runtime if needed |
| Flat SARIMAX forecast | `m` not set or `D=0` | Ensure `m=4` in `auto_arima` |
| Prophet flat forecast | Wrong seasonality mode | Try `"additive"` instead of `"multiplicative"` |
| `LinAlgError` | Edge-case parameters | Use `enforce_stationarity=False, enforce_invertibility=False` |
| Rolling window starts with NaN | Expected behavior | First `window-1` values are NaN; use `.dropna()` for calculations |

---
<p style="color:#7F8C8D; font-size:0.85em;">
<em>CAP4767 Data Mining with Python | Miami Dade College | Spring 2026</em><br>
Lab 1 — Time Series Exploration & Forecasting (Chapters 1–2) | 50 Points
</p>